In [ ]:
# Model building
import torch
import torchvision
import torchvision.transforms as tf # data augmentation and resizing
import torchvision.models as models # to get pretrained models
import torch.nn as nn # to build NN, criterion
import torch.optim as optim # optimizer

# plotting and evaluation
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix as conf_mat # performance evaluation

# Datapipe Line
import pandas as pd # read csv
from imblearn.over_sampling import RandomOverSampler as ROS # training data oversampling
from sklearn.model_selection import train_test_split # splitting dataframes
from torch.utils.data import Dataset, DataLoader # data pipeline

# utils
import numpy as np
import os
import random
import torch.nn.functional as F # softmax
from PIL import Image
from tqdm import tqdm

# Setting device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


# For reproducibility
RANDOM_SEED = 42 

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(RANDOM_SEED)

In [ ]:
lr = 0.001
bs = 128 # On P100, 512 is okay for resnet32 and resnet50, bs 256 is okay for densenet121 and densenet201
EPOCHS = 40 # number of training epochs in total
FT_EPOCHS = 5 # first epochs to fine tune classifier head only, after that end to end finetuning begins
LR_MIN = 1e-6 # minimum learning rate, below that, early training stopping occurs

highest_acc = 0.80 # Manually set for now

#Choose CNN backbone
CNNS = ['resnet34', 'resnet50', 'densenet121', 'densenet201']
CHOSEN_MODEL = CNNS[2] #desnenet121 performs best

# Flags
SAVE_CHECKPOINT = True
SAVE_HIGHEST = True

In [ ]:
data = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv')
data.head()
classes = {0: ('akiec', 'Actinic keratoses'),  
           1:('bcc' , ' basal cell carcinoma'), 
           2:('bkl', 'benign keratosis-like lesions'), 
           3: ('df', 'dermatofibroma'),
           4: ('nv', ' melanocytic nevi'), 
           5: ('vasc', ' pyogenic granulomas and hemorrhage'), 
           6: ('mel', 'melanoma'),
          }

CLASSES = [classes[idx][0] for idx in range(len(classes))] #abbreviated form of classes
CLASSES_FULL = [classes[idx][1] for idx in range(len(classes))] #Full name of classes
CLASSES, CLASSES_FULL

In [ ]:
#we divide the data from the labels to use the train_test_split function, then we will add the labels back to an overall dataframe so it is easy to 
#associate the data with its label 
x = data.drop(labels = 'label', axis = 1) #get image data without labels
y = data.label #just labels

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42) #split data into training and testing (also shuffles data)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=1) #split the remaining data into val and test

#we use a 80% train, 10% validation, 10% test split
print('% of images used for training: {}%:'.format(100*np.shape(x_train)[0]/10000))
print('% of images used for testing: {}%'.format(100*np.shape(x_test)[0]/10000))
print('% of images used for validation: {}%'.format(100*np.shape(x_val)[0]/10000))

#randomly oversample the training data to avoid bias in learning process
randOvrSamp = ROS()
x_train, y_train = randOvrSamp.fit_resample(x_train, y_train)


#add the labels to the overall training dataframex
#we copy xtrain here to avoid fragmentation from the previous inserts
x_train = x_train.copy()
x_train.insert(0, 'label', value = y_train.values)
trainingData = x_train

#add the labels to the overall validation dataframe
x_val.insert(0, 'label', value = y_val.values)
valData = x_val

#add the labels to the overall testing dataframe
x_test.insert(0, 'label', value = y_test.values)
testData = x_test

In [ ]:
#we only run this cell to calculate the mean and std deviation, we don't need to run it once we know these values

#we'll calculate the mean and std deviation for normalization here:
temp = x

#reshape the array to be (# of images * w * l * 3)
temp = np.asarray(temp).reshape(-1, 28, 28, 3)/255

#mean
mean = torch.tensor([np.mean(temp[:,:,:,channel]) for channel in range(3)])
mean = torch.flatten(mean.reshape(1,3,1,1))
print(mean)

#std deviation
std = torch.tensor([np.std(temp[:,:,:,channel]) for channel in range(3)])
std = torch.flatten(std.reshape(1,3,1,1))
print(std)



Define the CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes, model = 'resnet50'):
        super(CNN, self).__init__()
        self.num_classes = num_classes
        
        self.chosen_model = model
        if self.chosen_model == 'resnet50':
            self.model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            self.classifier = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(self.model.fc.in_features, 256, bias=False),
                nn.ReLU(),
                nn.BatchNorm1d(256),
                
                nn.Linear(256, 256//2, bias=False),
                nn.ReLU(),
                nn.BatchNorm1d(256//2),
                
                nn.Linear(128, self.num_classes, bias=False),
                nn.BatchNorm1d(self.num_classes),
            )
        self.model.fc = self.classifier
    def forward(self,x):
        return self.model(x)
    

In [ ]:
class HAM10K(Dataset):
    def __init__(self, dataframe, transforms = None):
        self.data = dataframe #all data
        self.y = self.data.label #ground truth
        self.x = self.data.drop(labels = 'label', axis = 1) #image data ie. input 
        
        #reshape data to be 28x28 3 channel images
        self.x = np.asarray(self.x, dtype=np.uint8).reshape(-1, 28,28,3)
        self.tf = transforms
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        label = torch.tensor(self.y.iloc[idx])
        img = self.x[idx]
        img =  Image.fromarray(img, 'RGB')
        if self.tf != None:
            img = self.tf(img)
        return img, label
        

In [ ]:
train_transforms = tf.Compose(
    [
        
        tf.Resize(232),
        tf.CenterCrop(224),
        tf.RandomVerticalFlip(0.5),
        tf.RandomHorizontalFlip(0.5),
        tf.RandomRotation(0.2),
        tf.ToTensor(),
        tf.Normalize([0.7636, 0.5462, 0.5706], [0.1391, 0.1504, 0.1673])
    ]
)

val_transforms = tf.Compose(
    [
        tf.Resize(232),
        tf.CenterCrop(224),
        tf.ToTensor(),
        tf.Normalize([0.7636, 0.5462, 0.5706], [0.1391, 0.1504, 0.1673])
    ]

)

test_transforms = None


train_ds = HAM10K(trainingData, train_transforms)
trainLoader = DataLoader(train_ds, batch_size = bs, shuffle = True, pin_memory = True, num_workers = 2)

val_ds = HAM10K(valData, train_transforms)
valLoader = DataLoader(val_ds, batch_size = bs, shuffle = False, pin_memory = True, num_workers = 2)

test_ds = HAM10K(testData, train_transforms)
testLoader = DataLoader(test_ds, batch_size = bs, shuffle = False, pin_memory = True, num_workers = 2)

In [ ]:
height = 5
width = 5
fig = plt.figure(figsize = (10,10))
for i in range(25):
    trainim, label= train_ds.__getitem__(i)
    trainim = trainim.permute(2,1,0)
    fig.add_subplot(height, width, i+1)
    plt.imshow(trainim)
    plt.axis('off')
    plt.title('Class {}'.format(CLASSES[label]))

Training Loop


In [ ]:
model = CNN(len(CLASSES), model = 'resnet50').to(DEVICE)

#freeze all weights
for p in model.parameters():
    p.requires_grad = False

#only want to update the classifier weights
for p in model.classifier.parameters():
    p.requires_grad = True

optimizer = optim.AdamW(model.parameters(), lr = lr, weight_decay = 0.1)

criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True) 

In [ ]:
train_acc = []
train_losses = []

test_acc = []
test_losses = []

lrs = []

# For warmup
lr0 = lr*0.01

lr_step = (lr-lr0)/(len(trainLoader)-1) # lr step for slow warm up from lr0 to lr 

print(f'Highest acc to beat: {highest_acc}')
best_acc = 0.90*highest_acc # use to decide whether to save a run 

for epoch in range(EPOCHS):
        loader = tqdm(trainLoader)
        losses = [] # logs avg loss per epoch
        accs = [] # logs avg acc per epoch
        correct = 0 # counts how many correct predictions
        count = 0 # counts how many samples
        
        if epoch>0:
            lrs+= [optimizer.param_groups[0]['lr']]*len(trainLoader) # track lr after warm up
        
        if epoch==FT_EPOCHS:
            for p in model.parameters():
                p.requires_grad = True
            print('End to End Fine Tuning Begins')

        model.train()
        for bidx, (images, labels) in enumerate(loader):
            
            # Warm up, slowly increase fomr lr0 to lr in first epoch
            if epoch==0:
                    
                lr_ = lr0 + lr_step*bidx

                for op in optimizer.param_groups:
                    op['lr'] = lr_

                lrs.append(optimizer.param_groups[0]['lr']) # track lr

            images = images.to(DEVICE) # move to gpu
            labels = labels.to(DEVICE)
                
            score = model(images)
            loss = criterion(score, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                pred = torch.argmax(score, -1).detach() # need to detach else bug
                correct += (pred==labels).sum() # count how many correct
                count += len(labels)
                acc = correct/count # accumated accuracy
                
                losses.append(loss.item())
                accs.append(acc)
                
                loader.set_description(f'TRAIN | epoch {epoch+1}/{EPOCHS} | acc {acc:.4f} | loss {loss.item():.4f}')
                
        train_acc.append(acc)
        train_losses.append(torch.tensor(losses).mean().item())
                
        model.eval()
        with torch.no_grad():
            
            loader = tqdm(valLoader)
            
            losses = [] # logs loss per minibatch
            accs = [] # logs running acc throughout one epoch
            
            correct = 0 # counts how many correct predictions in one epoch
            count = 0 # counts how many samples seen in one epoch
            
            for bidx, (images, labels) in enumerate(loader):
                images = images.to(DEVICE) # move to gpu
                labels = labels.to(DEVICE)

                score = model(images)
                loss = val_criterion(score, labels)
                
                pred = torch.argmax(score, -1).detach() # need to detach else bug
                correct += (pred==labels).sum() # count how many correct
                count += len(labels)

                acc = correct/count # accumated accuracy
                loader.set_description(f'TEST | epoch {epoch+1}/{EPOCHS} | acc {acc:.4f} | loss {loss.item():.4f}')
                
                losses.append(loss.item())
                accs.append(acc)

        test_acc.append(acc)
        test_losses.append(torch.tensor(losses).mean().item())
        
        scheduler.step(torch.tensor(acc)) # reduce lr if test acc does not improve
        
        if SAVE_CHECKPOINT==True:
            if test_acc[-1]>best_acc:
                best_acc = test_acc[-1].item()

                checkpoint = {
                    'model': model,
                    'losses': [train_losses, test_losses],
                    'accs': [train_acc, test_acc],
                    'lrs': lrs,
                    'best_acc': best_acc,
                    'last_epoch_trained': epoch,
                }
                
                if best_acc > highest_acc and SAVE_HIGHEST==True:
                    old_highest_acc = highest_acc
                    highest_acc = best_acc
                    torch.save(highest_acc, 'highest_acc.pt')
                    print(f'HIGHEST ACCURACY SURPASSED from {old_highest_acc} to {highest_acc}') # save highest achieving model evahhh (yayyy! ^.^)
                    torch.save(checkpoint, f'{highest_acc:.4f} checkpoint.pt')

                torch.save(checkpoint, 'checkpoint.pt')
                print(f'Checkpointed with {best_acc:.4f} best acc')
            
        if optimizer.param_groups[0]['lr']<LR_MIN:
            print(f'EARLY STOPPING! LR below {LR_MIN}')
            break